# ⚛️ LiH — Noiseless VQE Ground-State (UCCSD)

This notebook computes the **ground-state energy of LiH** using the production
VQE API:

    from vqe.core import run_vqe

### Simulation settings
- Molecule: **LiH**
- Basis: **STO-3G**
- Ansatz: **UCCSD** (singles + doubles)
- Optimizer: **Adam**
- Device: `default.qubit`
- Steps: 60

This is a **pure package client**:  
no manual QNodes, no excitations, no custom ansatz building, no caching code.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

# Allow local import when running from notebooks/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe
from vqe_qpe_common.molecules import get_molecule_config
from vqe_qpe_common.plotting import IMG_DIR

# 🧬 Molecular System — LiH

We load LiH from the shared molecule registry.

In [ ]:
mol = get_molecule_config("LiH")

symbols = mol["symbols"]
coordinates = mol["coordinates"]
basis = mol["basis"]
charge = mol["charge"]

print("LiH molecular specification:")
print(mol)

# 🚀 Run UCCSD VQE (Noiseless)

`run_vqe` handles:
- UCCSD ansatz construction  
- Hamiltonian generation  
- Optimizer creation  
- Device management  
- Caching  
- Statevector extraction  

In [ ]:
result = run_vqe(
    molecule="LiH",
    ansatz_name="UCCSD",
    optimizer_name="Adam",
    n_steps=60,
    stepsize=0.2,
    noisy=False,
    plot=False,
    seed=0,
    force=False,
)

energies = result["energies"]
final_energy = result["energy"]

print(f"\nFinal UCCSD energy for LiH: {final_energy:.8f} Ha")

# 📉 Convergence Plot

In [ ]:
plt.figure(figsize=(9, 5))
plt.plot(energies, marker="o")
plt.xlabel("Iteration")
plt.ylabel("Energy (Ha)")
plt.title("LiH Ground-State VQE Convergence (UCCSD, Noiseless)")
plt.grid(True, alpha=0.4)
plt.tight_layout()

fname = os.path.join(IMG_DIR, "LiH_UCCSD_Noiseless_Convergence.png")
plt.savefig(fname, dpi=300)
plt.show()

print(f"Saved convergence plot to:\n  {fname}")

# 🔍 Final Statevector: Significant Components

We reconstruct the wavefunction and display basis states with amplitude:

\[
|\psi_i| > 10^{-2}
\]

In [ ]:
def reconstruct_state(rec):
    """Return complex statevector from stored real/imag arrays."""
    psi_r = np.array(rec["final_state_real"])
    psi_i = np.array(rec["final_state_imag"])
    return psi_r + 1j * psi_i

psi = reconstruct_state(result)
num_qubits = result["num_qubits"]

threshold = 1e-2
nz = np.where(np.abs(psi) > threshold)[0]

print("\nSignificant components of |ψ>:")
terms = []
for idx in nz:
    amp = psi[idx]
    bit = f"|{idx:0{num_qubits}b}⟩"
    if abs(amp.imag) < 1e-6:
        s = f"{amp.real:.4f}{bit}"
    else:
        s = f"({amp.real:.4f}+{amp.imag:.4f}j){bit}"
    terms.append(s)

print(" |ψ⟩ =", " + ".join(terms))

# 📊 Amplitude Bar Plot (Optional)
Displays the amplitudes of significant basis states.

In [ ]:
magnitudes = np.abs(psi[nz])
labels = [f"|{i:0{num_qubits}b}⟩" for i in nz]

plt.figure(figsize=(10, 5))
plt.bar(labels, magnitudes)
plt.xlabel("Basis state")
plt.ylabel("Amplitude")
plt.title("LiH Ground-State Amplitudes (UCCSD, Noiseless)")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()

fname = os.path.join(IMG_DIR, "LiH_UCCSD_Statevector.png")
plt.savefig(fname, dpi=300)
plt.show()

print(f"Saved statevector amplitude plot to:\n  {fname}")